This notebook filters the subcortical regions of Lausanne463, computes Euclidean distance and zip everything for use by TVB.
Then 5HT2a vector's subcortical regions are also filtered out.

In [8]:
from scipy.io import loadmat
import numpy as np
import zipfile
import os

In [4]:
path = './data/connectivity/Lausanne463'
sub = loadmat(f'{path}/Lausanne_463_subnetworks.mat')['subnetworks']
cortical = np.where(np.isin(sub, [1,2,3,4,5,6]))[0]
weights = loadmat(f'{path}/Lausanne463_HCP_DTI_count.mat')['connectivity']
coords = np.loadtxt(f'{path}/Lausanne_463_MNI_coords.node')
print(coords.shape)

(463, 5)


In [5]:
# Filter subcortical regions and shape coordinates for TVB (with id, x, y, z for each region)
weights_filtered = weights[cortical][:,cortical]
coords_cort = coords[cortical,:3]
coords_filtered = np.concatenate((np.arange(len(cortical)).reshape(-1,1), coords_cort), axis=1)

In [6]:
# Generate tract lengths approximation using Euclidean distance
tl = np.zeros((len(cortical),len(cortical)))
for i in range(len(cortical)):
    for j in range(len(cortical)):
        tl[i,j] = np.linalg.norm(coords_filtered[i] - coords_filtered[j])

In [7]:
# Save each file individually
np.savetxt(f'{path}/centers.txt', coords_filtered, fmt='%i')
np.savetxt(f'{path}/weights.txt', weights_filtered, fmt='%f')
np.savetxt(f'{path}/tract_lengths.txt', tl, fmt='%f')

In [10]:
# Zip files for TVB
files = [
    f'{path}/centers.txt',
    f'{path}/weights.txt',
    f'{path}/tract_lengths.txt'
]
zip_path = f'{path}/connectivity_cortical_Lausanne463.zip'

with zipfile.ZipFile(zip_path, 'w') as zipf:
    for file in files:
        zipf.write(file, arcname=os.path.basename(file))

print(f"Zip file created at: {zip_path}")

Zip file created at: ./data/connectivity/Lausanne463/connectivity_cortical_Lausanne463.zip


In [11]:
receptors = loadmat(f'{path}/5HTvecs_ls463.mat')

In [12]:
fht2a = receptors['mean5HT2A_ls463'].flatten()[cortical]
fht1a = receptors['mean5HT1A_ls463'].flatten()[cortical]

In [14]:
np.savetxt(f'{path}/5HT2a_Lausanne463_cortical.txt', fht2a, fmt='%f')
np.savetxt(f'{path}/5HT1a_Lausanne463_cortical.txt', fht1a, fmt='%f')